In [1]:
# Load library
import csv
import pandas as pd
import numpy as np
import time
# from datetime import datetime, timedelta
import datetime as dt

In [65]:
# Read data full 2005 - 2015
data_2005 = pd.read_csv("data/data_odd_2005_regression.csv", encoding='utf-8')
data_2016 = pd.read_csv("data/data_EURO_2016.csv", encoding='utf-8')

# Convert date to Datetime object
data_2005['date'] = pd.to_datetime(data_2005['date'])
data_2016['date'] = pd.to_datetime(data_2016['date'])

In [42]:
data_2005.columns

Index(['id', 'date', 'team_1', 'team_2', 'home_team', 'tournament',
       'h_win_diff', 'h_draw', 'f_goalF_1', 'f_goalF_2', 'f_goalA_1',
       'f_goalA_2', 'f_win_1', 'f_win_2', 'f_draw_1', 'f_draw_2',
       'avg_odds_win_1', 'avg_odds_draw', 'avg_odds_win_2', 'result',
       'goal_diff', 'f_diff_goalF', 'form_diff_goalF', 'form_diff_goalA',
       'form_diff_win', 'form_diff_draw', 'odd_diff_win', 'odd_draw'],
      dtype='object')

In [24]:
data_2016.head()

,id,date,team_1,team_2,home_team,tournament,h_win_diff,h_draw,f_goalF_1,f_goalF_2,...,f_goalA_2,f_win_1,f_win_2,f_draw_1,f_draw_2,avg_odds_t1_win,avg_odds_draw,avg_odds_t2_win,result,goal_diff
0,37400,2016-06-10,France,Romania,France,UEFA Euro,5,5,21,8,...,11,8,2,0,7,1.360,4.88,11.190,win,1
1,37401,2016-06-11,Albania,Switzerland,Albania,UEFA Euro,-5,1,10,19,...,13,4,5,3,0,5.130,3.25,1.909,lose,-1
2,37403,2016-06-11,England,Russia,England,UEFA Euro,2,4,15,15,...,16,8,6,0,0,1.769,3.57,5.500,draw,0
3,37407,2016-06-11,Slovakia,Wales,Wales,UEFA Euro,0,0,16,9,...,8,5,4,3,3,2.700,2.96,3.130,lose,-1
4,37412,2016-06-12,Germany,Ukraine,Germany,UEFA Euro,2,3,18,14,...,9,5,7,0,2,1.521,4.34,7.350,win,2


In [66]:
# Read data squad strength
ss_wc_10 = pd.read_csv("web_crawler/squad/squad_strength/2010_FIFA_World_Cup_squads_strength__.csv", encoding='utf-8')
ss_wc_14 = pd.read_csv("web_crawler/squad/squad_strength/2014_FIFA_World_Cup_squads_strength__.csv", encoding='utf-8')
ss_eu_12 = pd.read_csv("web_crawler/squad/squad_strength/2012_UEFA_Euro_squads_strength__.csv", encoding='utf-8')
ss_eu_16 = pd.read_csv("web_crawler/squad/squad_strength/2016_UEFA_Euro_squads_strength__.csv", encoding='utf-8')

ss_wc_10 = ss_wc_10.set_index(ss_wc_10['Nation'])
ss_wc_14 = ss_wc_14.set_index(ss_wc_14['Nation'])
ss_eu_12 = ss_eu_12.set_index(ss_eu_12['Nation'])
ss_eu_16 = ss_eu_16.set_index(ss_eu_16['Nation'])

In [175]:
# data_WC_2010_ = data_2005[(data_2005['tournament'] == 'FIFA World Cup') & (data_2005['date'].dt.year == 2010 )]
# data_WC_2010_.to_csv("data_WC_2010",index=False)

# data_WC_2014_ = data_2005[(data_2005['tournament'] == 'FIFA World Cup') & (data_2005['date'].dt.year == 2014 )]
# data_WC_2014_.to_csv("data_WC_2014",index=False)

# data_EUR_2012_ = data_2005[(data_2005['tournament'] == 'UEFA Euro') & (data_2005['date'].dt.year == 2012 )]
# data_EUR_2012_.to_csv("data_EURO_2012",index=False)

In [67]:
data_WC_2010_ = data_2005[(data_2005['tournament'] == 'FIFA World Cup') & (data_2005['date'].dt.year == 2010 )]
list_nation = ss_wc_10['Nation'].unique()
data_WC_2010 = data_WC_2010_[(data_WC_2010_['team_1'].isin(list_nation)) & (data_WC_2010_['team_2'].isin(list_nation))]

data_WC_2014_ = data_2005[(data_2005['tournament'] == 'FIFA World Cup') & (data_2005['date'].dt.year == 2014 )]
list_nation = ss_wc_14['Nation'].unique()
data_WC_2014 = data_WC_2014_[(data_WC_2014_['team_1'].isin(list_nation)) & (data_WC_2014_['team_2'].isin(list_nation))]

In [18]:
data_EUR_2012_ = data_2005[(data_2005['tournament'] == 'UEFA Euro') & (data_2005['date'].dt.year == 2012 )]
list_nation = ss_eu_12['Nation'].unique()
data_EUR_2012 = data_EUR_2012_[(data_EUR_2012_['team_1'].isin(list_nation)) & (data_EUR_2012_['team_2'].isin(list_nation))]

data_EUR_2016_ = data_2016.copy()
list_nation = ss_eu_16['Nation'].unique()
data_EUR_2016 = data_EUR_2016_[(data_EUR_2016_['team_1'].isin(list_nation)) & (data_EUR_2016_['team_2'].isin(list_nation))]

In [19]:
# Add feature 
# ff_Rival?
new_features= ["ff_diff_rank",
               "ff_diff_ovr",
               "ff_diff_attk",
               "ff_diff_mid",
               "ff_diff_def",
               "ff_diff_prestige",
               "ff_diff_age11",
               "ff_diff_ageAll",
               "ff_diff_bup_speed",
               "ff_diff_bup_pass",
               "ff_diff_cc_pass",
               "ff_diff_cc_cross",
               "ff_diff_cc_shoot",
               "ff_diff_def_press",
               "ff_diff_def_aggr",
               "ff_diff_def_teamwidth"]

squad_features = ['Rank',
                  'Overall',
                 'Attack',
                 'Midfield',
                  'Defence',
                  'InternationalPrestige',
                  'Starting11AverageAge',
                  'WholeTeamAverageAge',
                 'BuildUpPlay_Speed', 
                 'BuildUpPlay_Passing',
                 'ChanceCreation_Passing', 
                 'ChanceCreation_Crossing',
                 'ChanceCreation_Shooting', 
                 'Defence_Pressure', 
                 'Defence_Aggression', 
                 'Defence_TeamWidth']

In [20]:
def diff_feature(row,df):
    team1= row.team_1
    team2= row.team_2
    rank1 = df.loc[team1]
    rank2 = df.loc[team2]
    return (rank1-rank2)

        
def add_feature(df_event,df_squad):    
    for i in range(len(new_features)):
        new_feature = new_features[i]
        squad_feature = squad_features[i]
        df_event[new_feature] = df_event.apply(diff_feature,df = df_squad[squad_feature],axis=1)
        


In [21]:
add_feature(data_WC_2010,ss_wc_10)
add_feature(data_WC_2014,ss_wc_14)
add_feature(data_EUR_2012,ss_eu_12)
add_feature(data_EUR_2016,ss_eu_16)

c:\users\mrthl\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [22]:
data_EUR_2016.head()

,id,date,team_1,team_2,home_team,tournament,h_win_diff,h_draw,f_goalF_1,f_goalF_2,...,ff_diff_age11,ff_diff_ageAll,ff_diff_bup_speed,ff_diff_bup_pass,ff_diff_cc_pass,ff_diff_cc_cross,ff_diff_cc_shoot,ff_diff_def_press,ff_diff_def_aggr,ff_diff_def_teamwidth
0,37400,2016-06-10,France,Romania,France,UEFA Euro,5,5,21,8,...,0.09,-0.82,-7,-14,9,-13,17,1,1,2
2,37403,2016-06-11,England,Russia,England,UEFA Euro,2,4,15,15,...,-3.73,-3.34,-11,3,-6,6,-6,-4,0,-4
5,37415,2016-06-12,Northern Ireland,Poland,Poland,UEFA Euro,1,2,13,29,...,0.36,1.22,2,20,14,-2,6,3,-6,9
7,37417,2016-06-13,Belgium,Italy,Belgium,UEFA Euro,-9,4,18,16,...,-3.00,-2.39,-19,-18,-6,-27,4,24,12,16
9,37421,2016-06-13,Czech Republic,Spain,Spain,UEFA Euro,-3,1,21,16,...,-2.55,-0.56,6,12,15,15,-5,10,-3,7


In [23]:
data_WC_2010.to_csv("data\data_event\data_WC_2010.csv",index=False)
data_WC_2014.to_csv("data\data_event\data_WC_2014.csv",index=False)
data_EUR_2012.to_csv("data\data_event\data_EUR_2012.csv",index=False)
data_EUR_2016.to_csv("data\data_event\data_EUR_2016.csv",index=False)